In [1]:
from similarity_search_gui import load_images_features, find_similar_images
from misc import load, load_class_labels
from data_loader import data_loader;
import numpy as np
from tqdm import tqdm

dataset_root = "../dataset/"
test_info = load(dataset_root + 'labels/test.csv')
test_dir = dataset_root + 'val_set/'

dl_test = data_loader(test_info, test_dir, 1, (224, 224));

2024-01-12 09:07:12.008693: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-12 09:07:12.063437: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-12 09:07:12.326139: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 09:07:12.326270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 09:07:12.356775: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
nn, _ = load_images_features(False)

def evaluate(base_weight):
    results = []
    min_cut = 60;

    for i in tqdm(range(dl_test.number_of_batch())):
        _, labels, filenames = dl_test.get_batch(i);
        query_label = labels[0];
        query_filename = filenames[0];
        query_filename = dataset_root + 'complete/' + query_filename;

        most_similar_filenames = find_similar_images(query_filename, None, nn, False, True, base_weight=base_weight, verbose=0);
        most_similar_filenames = np.array([file.split('/')[-1] for file in most_similar_filenames]);
        mask = [('val' in file) for file in most_similar_filenames];
        most_similar_filenames = most_similar_filenames[mask];

        cut_point = min(len(test_info[test_info['label'] == query_label].values), min_cut)
        most_similar_filenames = most_similar_filenames[:cut_point];
        
        correct = 0
        wrong = 0 
        for file in most_similar_filenames:
            label_found = test_info[test_info['filename'] == file]['label'].values[0]
            if label_found == query_label:
                correct += 1;
            else:
                wrong += 1
        results.append((correct/(correct + wrong), (correct + wrong)))
        #if i % 10 == 0:
        #    print(correct, wrong)

        total_element = 0
        acc = 0
        for res in results:
            perc, weight = res;
            total_element += weight;
            acc += perc * weight;
        acc /= total_element
            
    return results, acc
        

## Evaluation with only efficient net base

In [3]:
res, acc = evaluate(1)
print(acc)

100%|██████████| 11994/11994 [2:37:03<00:00,  1.27it/s] 

0.1254561592190722


In [6]:
res = np.array(res)
np.save('../dataset/evaluation_efficient_net.npy', res);

In [5]:
print(res)

[(0.21818181818181817, 55), (0.2727272727272727, 55), (0.09090909090909091, 55), (0.10909090909090909, 55), (0.05454545454545454, 55), (0.2, 55), (0.3090909090909091, 55), (0.16363636363636364, 55), (0.12727272727272726, 55), (0.01818181818181818, 55), (0.41818181818181815, 55), (0.2909090909090909, 55), (0.23636363636363636, 55), (0.10909090909090909, 55), (0.07272727272727272, 55), (0.3090909090909091, 55), (0.01818181818181818, 55), (0.2545454545454545, 55), (0.03636363636363636, 55), (0.16363636363636364, 55), (0.32727272727272727, 55), (0.03636363636363636, 55), (0.05454545454545454, 55), (0.2545454545454545, 55), (0.01818181818181818, 55), (0.2, 55), (0.09090909090909091, 55), (0.01818181818181818, 55), (0.4727272727272727, 55), (0.07272727272727272, 55), (0.2545454545454545, 55), (0.5818181818181818, 55), (0.2909090909090909, 55), (0.2545454545454545, 55), (0.23636363636363636, 55), (0.2545454545454545, 55), (0.3090909090909091, 55), (0.01818181818181818, 55), (0.381818181818181

## Evaluation with ensembling model

In [7]:
res, acc = evaluate(0)
print(acc)

100%|██████████| 11994/11994 [4:37:17<00:00,  1.39s/it] 

0.3988849161673733


In [8]:
res = np.array(res)
np.save('../dataset/evaluation_ensembling.npy', res);